In [31]:
import pandas as pd
import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math


#read the file
    
data = pd.read_table('6119-hybridOut-June17-27.txt', skiprows=1)

#drop columns with NaN

data = data.dropna(axis='columns', how ='all')

#fix the headers

data = data.rename({'Event': 'Event', 'MinFracDay': 'NmHitDAQ1', \

                   'JulDay':'MinFracDay','Unnamed: 6':'JulDay', \

                   'Unnamed: 8':'SSDB', 'Unnamed: 10':'Hit1.ch1', \

                   'Unnamed: 13':'Hit1.ch2','Unnamed: 16':'Hit1.ch3', \

                   'Unnamed: 19':'Hit1.ch4','Unnamed: 22':'Hit2.ch1', \

                   'Unnamed: 25':'Hit2.ch2','Unnamed: 28':'Hit2.ch3', \

                   'Unnamed: 31':'Hit2.ch4'

                   },

                  axis=1)
cols = data.columns

cols = cols.map(lambda x: x.replace('#', '_') if isinstance(x, (str or unicode)) else x)

data.columns = cols

data.head(3)

,Event,NmHitDAQ1,MinFracDay,JulDay,SSDB,eventDateTime,Hit1.ch1,PW1,Hit1.ch2,PW2,...,Hit1.ch4,PW4,Hit2.ch1,PW1.1,Hit2.ch2,PW2.1,Hit2.ch3,PW3.1,Hit2.ch4,PW4.1
0,1,4,0.500012,2459747,43201.03,6/17/2022 0:0:1,0.00,17.50,2.50,25.00,...,3.0,25.01,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,2,4,0.500012,2459747,43201.07,6/17/2022 0:0:1,0.75,22.49,0.75,23.75,...,0.0,23.75,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,3,2,0.500013,2459747,43201.16,6/17/2022 0:0:1,-1.00,-1.00,-1.00,-1.00,...,0.0,32.50,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [32]:
data['Filter_1'] = data.iloc[:, 6] != -1  
data['Filter_2'] = data.iloc[:, 8] != -1
data['Filter_3'] = data.iloc[:, 10] != -1 
data['Filter_4'] = data.iloc[:, 12] != -1 

#data.head(5)

filter_to_data = data.query('Filter_1 == True and Filter_3 == True and Filter_2 == True and Filter_4 == True')

#Change the Hit Columns 
filter_to_data['Delta_T'] = (filter_to_data['Hit1.ch3'] - filter_to_data['Hit1.ch2'])


<ipython-input-32-cf1178bf6c8a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_to_data['Delta_T'] = (filter_to_data['Hit1.ch3'] - filter_to_data['Hit1.ch2'])


In [40]:
channel_1 = 2.179
channel_2 = 1.697
channel_3 = 0.032
channel_4 = 0.758
speed_of_light_constant = 0.298

#Change channel numbers
channel_distance = abs(channel_3 - channel_2)


#Filter out outliers in calculations
no_outlier_data = list(filter(lambda x: x < 30 and x > -30, filter_to_data['Delta_T'])) 
mean = np.mean(no_outlier_data)
sigma = np.std(no_outlier_data)

#Limit Calculations
calculation_1 =(channel_distance / speed_of_light_constant)
calculation_2 = round(calculation_1 + calculation_1)
calculation_3 = mean - calculation_2


#You Can Change Sigma Cut
limit_1 = calculation_3 + (2 * sigma)
limit_2 = calculation_3 - (2 * sigma)

print("Limit 1 " + str(limit_1) + " (nanoseconds)")
print("Limit 2 " + str(limit_2) + " (nanoseconds)")


Limit 1 -3.3157375574237733 (nanoseconds)
Limit 2 -13.050569036356471 (nanoseconds)
